In [1]:
import numpy as np
from datetime import datetime

from env import MufexKeys
from strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest

from quantfreedom.exchanges.mufex_exchange.mufex import Mufex

mufex_main = Mufex(
    api_key=MufexKeys.mainnet_api_key,
    secret_key=MufexKeys.mainnet_secret_key,
    use_test_net=False,
)
symbol = 'BTCUSDT'

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

ValueError: field 'available_balance' occurs more than once

In [ ]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=3000,
)

In [ ]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.array([100]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

In [ ]:
backtest_settings = BacktestSettings()

mufex_main.set_exchange_settings(
    leverage_mode=LeverageModeType.Isolated,
    position_mode=PositionModeType.HedgeMode,
    symbol=symbol,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_available_balance=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_trades=np.array([0]),
    account_pct_risk_per_trade=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [ ]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

In [ ]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

In [ ]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=4,
    plot_results=True,
    logger_bool=False,
)

In [ ]:
order_records_df